<a href="https://colab.research.google.com/github/engineeringjoy/SynAnalyzer/blob/main/SynAnalyzer_ConvertImarisStatsFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SynAnalyzer.ConvertImarisStatsFile.ipynb
Created by: JFranco | 4 JUL 2024

This Python notebook is first module in the Synapse Analyzer pipeline. The purpose of the notebook is to take Excel files that were generated by Imaris, containing the statistics of all surfaces, and convert them into a simplified format that can be used by SynAnalyzer.ijm. The default values that are extracted are:


1. Surface ID as assigned by Imaris  
2. XYZ Positions
3. Volume for each surface

This code requires that the .xls files are in the original format output by Imaris. If "Spots" were used in Imaris, rather than surfaces, the code relating to volume extraction will need to adjusted.

In [ ]:
import numpy as np
import pandas as pd
import glob

In [ ]:
# Generate a list of all xls files added by the user
files = glob.glob('*.xls')
print(files)

['WSS_031.01.T3.01.Zs.4C.XYZ.PostSyn.xls']


In [ ]:
# Iterate through the xls files, extract the relavent sheet, and reformat
for file in files:
  # Read in the sheets that correspond to position and volume as separate dfs
  dfXYZ = pd.read_excel(file,skiprows=1, sheet_name='Position')
  dfVol = pd.read_excel(file,skiprows=1, sheet_name='Volume')
  # Reformat each df to make it a proper df rather than sheet
  dfXYZ = dfXYZ.drop(columns=['Unit', 'Category', 'Collection', 'Time'])
  dfXYZ.set_index('ID', inplace=True)
  dfVol = dfVol.drop(columns=['Unit', 'Category','Time'])
  dfVol.set_index('ID', inplace=True)
  # Need to transfer the information about the volume over the XYZ df without
  #. assumptions about df order
  for id in list(dfXYZ.index.values):
    # Get the volume associated with this ID from the volume df
    vol = dfVol['Volume'][id]
    # Store the volume in the XYX df
    dfXYZ.at[id, 'Volume_um3'] = vol

  dfXYZ.to_csv(file.split('.x')[0]+'.csv')